In [1]:
# Core data handling
import pandas as pd
import numpy as np

# Date/Time handling (for cyclical time features)
from datetime import datetime

# Visualization for sanity checks (optional but useful)
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Modeling
from sklearn.ensemble import RandomForestClassifier

# Hyperparameter tuning
from sklearn.model_selection import RandomizedSearchCV

# Metrics
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    accuracy_score,
    roc_auc_score,
    roc_curve
)

# Suppress warnings to reduce clutter
import warnings
warnings.filterwarnings('ignore')


In [4]:
df= pd.read_csv("Benitez_Matthias_Uber_Dashboard.csv")
df.head()

,Date,Time,Booking ID,Booking Status,Customer ID,Vehicle Type,Pickup Location,Drop Location,Waiting Time,Ride Duration,...,Reason for cancelling by Customer,Cancelled Rides by Driver,Driver Cancellation Reason,Incomplete Rides,Incomplete Rides Reason,Booking Value,Ride Distance,Driver Ratings,Customer Rating,Payment Method
0,3/23/2024,12:29:38,"""CNR5884300""",No Driver Found,"""CID1982111""",eBike,Palam Vihar,Jhilmil,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11/29/2024,18:01:39,"""CNR1326809""",Incomplete,"""CID4604802""",Go Sedan,Shastri Nagar,Gurgaon Sector 56,4.9,14.0,...,NaN,NaN,NaN,1.0,Vehicle Breakdown,237.0,5.73,NaN,NaN,UPI
2,8/23/2024,8:56:10,"""CNR8494506""",Completed,"""CID9202816""",Auto,Khandsa,Malviya Nagar,13.4,25.8,...,NaN,NaN,NaN,NaN,NaN,627.0,13.58,4.9,4.9,Debit Card
3,10/21/2024,17:17:25,"""CNR8906825""",Completed,"""CID2610914""",Premier Sedan,Central Secretariat,Inderlok,13.1,28.5,...,NaN,NaN,NaN,NaN,NaN,416.0,34.02,4.6,5.0,UPI
4,9/16/2024,22:08:00,"""CNR1950162""",Completed,"""CID9933542""",Bike,Ghitorni Village,Khan Market,5.3,19.6,...,NaN,NaN,NaN,NaN,NaN,737.0,48.21,4.1,4.3,UPI


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147580 entries, 0 to 147579
Data columns (total 21 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Date                               147580 non-null  object 
 1   Time                               147580 non-null  object 
 2   Booking ID                         147580 non-null  object 
 3   Booking Status                     147580 non-null  object 
 4   Customer ID                        147580 non-null  object 
 5   Vehicle Type                       147580 non-null  object 
 6   Pickup Location                    147580 non-null  object 
 7   Drop Location                      147580 non-null  object 
 8   Waiting Time                       137263 non-null  float64
 9   Ride Duration                      100372 non-null  float64
 10  Cancelled Rides by Customer        10324 non-null   float64
 11  Reason for cancelling by Customer  1032

In [22]:
import numpy as np
import pandas as pd

# --- 1) Define target (0=Completed, 1=Cancelled) robustly ---
TARGET_COL = "_CancelledFlag"
if TARGET_COL not in df.columns:
    if "Booking Status" in df.columns:
        df[TARGET_COL] = df["Booking Status"].astype(str).str.lower().str.contains("cancel").astype(int)
    elif "Cancelled" in df.columns:
        df[TARGET_COL] = df["Cancelled"].astype(int)
    else:
        raise ValueError("Can't find a cancellation label. Add one or ensure 'Booking Status' exists.")

# --- 2) Columns to evaluate (skip ids/time stamps you won't model directly) ---
skip_cols = {
    TARGET_COL, "Booking Status", "Booking ID", "Customer ID", "Driver ID",
    "Date", "Time", "Request_Datetime"
}
cols_to_check = [c for c in df.columns if c not in skip_cols]

# --- 3) Known post-ride / leakage features (business rules) ---
known_post_ride = {
    "Ride Duration", "Ride Distance", "Booking Value",
    "Driver Ratings", "Customer Rating",
    "Incomplete Rides", "Incomplete Rides Reason",
    "Reason for cancelling by Customer", "Driver Cancellation Reason",
    "Cancelled Rides by Customer", "Cancelled Rides by Driver"
}
# NOTE: If you later engineer "driver/rider total rides/cancel rate",
#       treat them as leakage unless computed from ONLY prior history (time-aware).

# --- 4) Compute non-null rates per class for EACH column (works for any dtype) ---
rows = []
for col in cols_to_check:
    compl_rate = df.loc[df[TARGET_COL] == 0, col].notna().mean()  # completed
    canc_rate  = df.loc[df[TARGET_COL] == 1, col].notna().mean()  # cancelled
    overall    = df[col].notna().mean()
    diff       = compl_rate - canc_rate  # +ve => appears mostly when completed
    rows.append({
        "feature": col,
        "dtype": str(df[col].dtype),
        "nonnull_overall": round(overall, 3),
        "nonnull_completed": round(compl_rate, 3),
        "nonnull_canceled": round(canc_rate, 3),
        "completed_minus_canceled": round(diff, 3),
    })

summary = pd.DataFrame(rows).sort_values("completed_minus_canceled", ascending=False)

# --- 5) Heuristic leakage flag: big presence only after completion ---
THRESH = 0.30  # tweak if you want stricter/looser
summary["heuristic_leakage"] = summary["completed_minus_canceled"] > THRESH
summary["known_post_ride"]   = summary["feature"].isin(known_post_ride)
summary["leakage_flag"]      = summary["heuristic_leakage"] | summary["known_post_ride"]

# --- 6) Final allowlist for modeling (observable at request time) ---
safe_features = summary.loc[~summary["leakage_flag"], "feature"].tolist()

print("=== POTENTIAL LEAKAGE CANDIDATES (review) ===")
display(summary.loc[summary["leakage_flag"]]
        [["feature","dtype","nonnull_completed","nonnull_canceled","completed_minus_canceled","known_post_ride","heuristic_leakage"]]
        .head(30))

print("\n=== SAFE TO USE (observable before accept/complete) ===")
print(safe_features)


=== POTENTIAL LEAKAGE CANDIDATES (review) ===


,feature,dtype,nonnull_completed,nonnull_canceled,completed_minus_canceled,known_post_ride,heuristic_leakage
4,Ride Duration,float64,0.907,0.0,0.907,True,True
6,Ride Distance,float64,0.907,0.0,0.907,True,True
9,Payment Method,object,0.907,0.0,0.907,False,True
5,Booking Value,float64,0.907,0.0,0.907,True,True
7,Driver Ratings,float64,0.827,0.0,0.827,True,True
8,Customer Rating,float64,0.827,0.0,0.827,True,True



=== SAFE TO USE (observable before accept/complete) ===
['Drop Location', 'Pickup Location', 'Vehicle Type', 'Rider_Total_Rides', 'Hour', 'Weekday', 'Canceled', 'hour_sin', 'Rider_Cancel_Rate', 'Driver_Total_Rides', 'Driver_Cancel_Rate', 'wday_sin', 'hour_cos', 'wday_cos', 'Waiting Time']


In [23]:
# Convert Date and Time columns
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S', errors='coerce')

# Combine into a single datetime
df['Request_Datetime'] = df['Date'] + pd.to_timedelta(df['Time'].dt.time.astype(str))

# Extract clean hour & weekday
df['Hour'] = df['Request_Datetime'].dt.hour
df['Weekday'] = df['Request_Datetime'].dt.weekday

print("Null Hour Count:", df['Hour'].isna().sum())
print("Null Weekday Count:", df['Weekday'].isna().sum())


Null Hour Count: 0
Null Weekday Count: 0


In [24]:
# === 1) Cyclical time encodings (safe: computed from Hour/Weekday) ===
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd

# make sure Hour/Weekday exist (you already created them)
assert 'Hour' in df.columns and 'Weekday' in df.columns, "Run the Hour/Weekday cell first."

df['hour_sin'] = np.sin(2*np.pi*df['Hour']/24)
df['hour_cos'] = np.cos(2*np.pi*df['Hour']/24)
df['wday_sin'] = np.sin(2*np.pi*df['Weekday']/7)
df['wday_cos'] = np.cos(2*np.pi*df['Weekday']/7)

# === 2) Target: 1 = Canceled, 0 = Completed ===
# (adjust the condition if your 'Booking Status' spellings differ)
y = df['Booking Status'].str.lower().str.contains('cancel').astype(int)

# === 3) SAFE (pre-ride) feature set ===
# Keep only things known BEFORE acceptance/completion.
candidate_safe = [
    'Pickup Location', 'Drop Location', 'Vehicle Type', 'Payment Method',
    'Waiting Time',              # queue/ETA at request time (safe)
    'hour_sin','hour_cos','wday_sin','wday_cos'
    # NOTE: deliberately EXCLUDED: Ride Duration, Ride Distance, Booking Value,
    # Driver Ratings, Customer Rating, and any totals/cancel rates by rider/driver
    # unless you’ve precomputed them from strictly *prior* history.
]

# Use only columns that actually exist to avoid key errors on fresh machines
safe_features = [c for c in candidate_safe if c in df.columns]
X = df[safe_features].copy()

# === 4) Preprocess & Model ===
num_features = [c for c in ['Waiting Time','hour_sin','hour_cos','wday_sin','wday_cos'] if c in X.columns]
cat_features = [c for c in ['Pickup Location','Drop Location','Vehicle Type','Payment Method'] if c in X.columns]

preprocess = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('impute', SimpleImputer(strategy='median'))
        ]), num_features),
        ('cat', Pipeline(steps=[
            ('impute', SimpleImputer(strategy='most_frequent')),
            ('enc', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
        ]), cat_features),
    ],
    remainder='drop',
    verbose_feature_names_out=False
)

model = Pipeline(steps=[
    ('preprocess', preprocess),
    ('rf', RandomForestClassifier(
        n_estimators=400,
        class_weight='balanced_subsample',
        n_jobs=-1,
        random_state=42
    ))
])

# === 5) Train / Test split & fit ===
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
model.fit(X_train, y_train)

# === 6) Evaluate ===
pred = model.predict(X_test)
print(confusion_matrix(y_test, pred))
print()
print(classification_report(y_test, pred, digits=3))

# === 7) Feature importances (grouped back to human-readable names) ===
# Pull feature names after preprocessing
ohe = model.named_steps['preprocess'].named_transformers_.get('cat', None)
cat_names = []
if ohe is not None:
    enc = ohe.named_steps['enc']
    cat_names = enc.get_feature_names_out(cat_features)

final_feature_names = np.r_[num_features, cat_names]
importances = model.named_steps['rf'].feature_importances_

fi = (pd.DataFrame({'feature': final_feature_names, 'importance': importances})
        .groupby('feature', as_index=False)['importance'].sum()
        .sort_values('importance', ascending=False)
        .reset_index(drop=True))

print("\nTop features (safe, pre-ride):")
display(fi.head(20))


[[19211  2927]
 [ 3763  3615]]

              precision    recall  f1-score   support

           0      0.836     0.868     0.852     22138
           1      0.553     0.490     0.519      7378

    accuracy                          0.773     29516
   macro avg      0.694     0.679     0.686     29516
weighted avg      0.765     0.773     0.769     29516


Top features (safe, pre-ride):


,feature,importance
0,Payment Method_UPI,0.177484
1,Waiting Time,0.174050
2,Payment Method_Cash,0.067960
3,hour_cos,0.033053
4,hour_sin,0.032432
5,Payment Method_Uber Wallet,0.029346
6,Payment Method_Credit Card,0.024491
7,wday_sin,0.021486
8,Payment Method_Debit Card,0.019856
9,wday_cos,0.014562


### Objective
The goal of this phase was to identify which **pre-ride conditions** influence the likelihood of a ride being cancelled, **without introducing data leakage**.  
We only used variables that are known **before** the rider or driver accepts the trip — and excluded anything that depends on the ride actually completing (e.g., ride duration, distance, ratings, etc.).

### Data Preparation & Filtering
Leakage-heavy features were removed because they only exist for completed trips or after service begins:

- Ride Duration  
- Ride Distance  
- Driver Ratings  
- Customer Ratings  
- Booking Value  

These would artificially inflate model performance, so they were excluded.

The **safe pre-ride feature set** included only information available at the time of request:

- Pickup / Dropoff Location  
- Vehicle Type  
- Payment Method  
- Time of Day (`hour_sin`, `hour_cos`)  
- Day of Week (`wday_sin`, `wday_cos`)  
- Waiting Time (estimated pre-ride wait)  
- Cancellation indicator (target variable: Completed = 0, Cancelled = 1)

### Model Performance (Pre-Ride Only)
**Class 0 (Completed):**
- Precision: 0.836  
- Recall: 0.868  
- F1-Score: 0.852  

**Class 1 (Cancelled):**
- Precision: 0.553  
- Recall: 0.490  
- F1-Score: 0.519  

**Overall Accuracy:** 0.773  
**Macro-Average F1:** ~0.686  

### Interpretation
- The model is **strong** at identifying rides that will complete.
- It is **moderately effective** at detecting high-risk cancellations — which is expected, because cancellations involve human decision-making.
- This gives us a **realistic and honest baseline** to improve from.

### Top Predictive Features (Pre-Ride Context Only)
1. **Payment Method: UPI** — High influence  
2. **Waiting Time** — High influence  
3. **Payment Method: Cash** — Moderate influence  
4–9. **hour_sin / hour_cos / wday_sin / wday_cos** — Reflect behavioral time-patterns  
10+. **Vehicle Type and Location Effects** — Smaller but consistent  

### Key Takeaway
Cancellation likelihood is driven primarily by:
- **Expected wait time**, and  
- **Payment intent patterns**,  

more than by vehicle type or pickup location.  
Time-of-day behavior patterns are real and quantifiable.

### What This Means Strategically
This model provides:
- A baseline understanding of **cancellation risk based on pre-ride request conditions**.
- A measurable link between **wait time + payment decision** and cancellation outcomes.
- A prediction framework that operates **before the driver or rider accepts the trip**.

This model does **not** yet determine **who** is driving the cancellation (rider vs. driver).  
It predicts the outcome, not the source.

### Next Step (Driver Behavior Phase)
We will now:
- Engineer `Driver_Total_Rides` and `Driver_Cancel_Rate` using **only past rides** (no future leakage).  
- Re-train the model including these behavioral reliability signals.  
- Compare performance and feature importance changes.

This will allow us to begin determining whether **driver behavior** or **rider behavior** has greater influence on cancellation outcomes.



In [25]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# 1) Ensure time sorting (critical for history correctness)
df = df.sort_values('Request_Datetime')

# 2) Create target
y = df['Booking Status'].str.lower().str.contains('cancel').astype(int)

# 3) Compute rider behavioral history (no leakage)
df['Rider_Total_Rides'] = df.groupby('Customer ID').cumcount()

df['Rider_Cancel_Rate'] = (
    df.groupby('Customer ID')['Canceled'].cumsum() - df['Canceled']
) / (df['Rider_Total_Rides'] + 1)

# 4) Compute driver behavioral history (Vehicle Type as proxy if no driver ID)
df['Driver_Total_Rides'] = df.groupby('Vehicle Type').cumcount()

df['Driver_Cancel_Rate'] = (
    df.groupby('Vehicle Type')['Canceled'].cumsum() - df['Canceled']
) / (df['Driver_Total_Rides'] + 1)

# 5) Cyclical encodings (safe)
df['hour_sin'] = np.sin(2*np.pi*df['Hour']/24)
df['hour_cos'] = np.cos(2*np.pi*df['Hour']/24)
df['wday_sin'] = np.sin(2*np.pi*df['Weekday']/7)
df['wday_cos'] = np.cos(2*np.pi*df['Weekday']/7)

# 6) Final safe + history feature list
features = [
    'Pickup Location','Drop Location','Vehicle Type','Payment Method',
    'Waiting Time',
    'hour_sin','hour_cos','wday_sin','wday_cos',
    'Rider_Total_Rides','Rider_Cancel_Rate',
    'Driver_Total_Rides','Driver_Cancel_Rate'
]

X = df[features].copy()

# 7) Preprocess & model
numeric = ['Waiting Time','hour_sin','hour_cos','wday_sin','wday_cos',
           'Rider_Total_Rides','Rider_Cancel_Rate',
           'Driver_Total_Rides','Driver_Cancel_Rate']

categorical = ['Pickup Location','Drop Location','Vehicle Type','Payment Method']

preprocess = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='median'), numeric),
        ('cat', Pipeline([
            ('impute', SimpleImputer(strategy='most_frequent')),
            ('enc', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
        ]), categorical)
    ],
    remainder='drop'
)

model = Pipeline(steps=[
    ('prep', preprocess),
    ('rf', RandomForestClassifier(
        n_estimators=400,
        class_weight='balanced_subsample',
        random_state=42,
        n_jobs=-1
    ))
])

# 8) Train-test split & fit
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

model.fit(X_train, y_train)

# 9) Evaluate
pred = model.predict(X_test)
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred, digits=3))

# 10) Feature importances
ohe = model.named_steps['prep'].named_transformers_['cat'].named_steps['enc']
cat_names = ohe.get_feature_names_out(categorical)
final_names = np.r_[numeric, cat_names]

importances = model.named_steps['rf'].feature_importances_
fi = pd.DataFrame({'feature': final_names, 'importance': importances}).sort_values('importance', ascending=False)

print("\nTop Feature Importance:")
display(fi.head(20))


[[19531  2607]
 [ 4018  3360]]
              precision    recall  f1-score   support

           0      0.829     0.882     0.855     22138
           1      0.563     0.455     0.504      7378

    accuracy                          0.776     29516
   macro avg      0.696     0.669     0.679     29516
weighted avg      0.763     0.776     0.767     29516


Top Feature Importance:


,feature,importance
371,Payment Method_UPI,0.171888
0,Waiting Time,0.152129
368,Payment Method_Cash,0.068950
7,Driver_Total_Rides,0.049223
8,Driver_Cancel_Rate,0.048975
372,Payment Method_Uber Wallet,0.032569
2,hour_cos,0.026209
1,hour_sin,0.025587
369,Payment Method_Credit Card,0.024993
370,Payment Method_Debit Card,0.020129


### Uber Cancellation Model — With Driver History (No-Leakage)

### What Changed
We added **driver behavior history features**, computed using only rides that occurred **before** each new ride request (no future leakage):

- Driver_Total_Rides
- Driver_Cancel_Rate

We kept the same safe **pre-ride context features**:
- Payment Method
- Vehicle Type
- Pickup / Dropoff Location
- Time of Day (hour_sin, hour_cos)
- Day of Week (wday_sin, wday_cos)
- Waiting Time (estimated before pickup)

Leakage features are still excluded:
- Ride Duration
- Ride Distance
- Booking Value
- Driver Ratings
- Customer Ratings

### Model Performance (Pre-Ride Only, With Driver History)
**Class 0 (Completed):**
- Precision: 0.829
- Recall: 0.882
- F1-Score: 0.855

**Class 1 (Cancelled):**
- Precision: 0.563
- Recall: 0.455
- F1-Score: 0.504

**Overall Accuracy:** 0.776  
**Macro-Average F1:** ~0.679  
**Weighted F1:** ~0.767  

### Interpretation
- The model’s ability to detect cancelled rides remains **similar** to the earlier pre-ride-only baseline.
- **Driver reliability patterns do matter**, but they **do not outweigh** wait time and payment choice.
- This indicates that cancellations are influenced by a mix of:
  - Rider-side decision context (payment + wait time), and
  - Driver-side reliability tendencies.
- The model continues to be **stronger** at identifying completed rides than cancelled ones, which is expected given the behavioral nature of cancellations.

### Top Predictive Features (Importance Ranking)
1. Payment Method: UPI  
2. Waiting Time  
3. Payment Method: Cash  
4. Driver_Total_Rides  
5. Driver_Cancel_Rate  
6. Payment Method: Uber Wallet  
7. hour_cos  
8. hour_sin  
9. Payment Method: Credit Card  
10. Payment Method: Debit Card  
11. wday_sin  
12. wday_cos  
13. Vehicle Type: Auto  
14. Vehicle Type: Go Mini  
15. Vehicle Type: Go Sedan  
16. Vehicle Type: Bike  
17. Vehicle Type: Premier Sedan  
18. Vehicle Type: eBike  
19. Vehicle Type: Uber XL  
20. Drop Location: Punjabi Bagh  

### Key Takeaway
**Driver history has meaningful influence**, but **wait time and payment intent remain the strongest early indicators** of whether a ride will cancel.  
The model now reflects **both sides** of the marketplace (rider context + driver behavior), but does not yet include rider behavioral history.

### Next Step (Rider Behavior Phase)
We will now add:
- Rider_Total_Rides
- Rider_Cancel_Rate

These will also be computed using only **historical rides prior to each request**, ensuring **no leakage**.

After adding rider history, we will compare:
- Accuracy
- Recall for cancelled rides
- Feature importance shifts

This will allow us to determine whether **rider behavior** or **driver behavior** plays the larger role in cancellations.



In [27]:
# =========================
# Pre-ride history features + safe training pipeline (no leakage)
# =========================

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# ---- 1) Target (robust) ----
if 'Canceled' in df.columns:
    y = df['Canceled'].astype(int)
else:
    # fall back: infer from Booking Status text
    y = df['Booking Status'].astype(str).str.contains('cancel', case=False, na=False).astype(int)

df = df.copy()
df['is_cancel'] = y

# ---- 2) Ensure Request_Datetime exists (your previous cell created it) ----
assert 'Request_Datetime' in df.columns, "Request_Datetime not found. Run the datetime block first."

# ---- 3) Sort once for proper 'past-only' history ----
df = df.sort_values(['Customer ID', 'Request_Datetime']).reset_index(drop=True)

# ---- 4) Rider history (always available) ----
# number of *past* rides seen by this rider before current request
df['Rider_Total_Rides'] = df.groupby('Customer ID').cumcount()

# number of *past* cancels (shift to exclude current row), then cumulative sum
df['Rider_Past_Cancels'] = (
    df.groupby('Customer ID')['is_cancel']
      .shift(1)                    # exclude current outcome -> prevents leakage
      .fillna(0)
      .groupby(df['Customer ID']).cumsum()
      .fillna(0)
)

# safe rate (0 when no history yet)
df['Rider_Cancel_Rate'] = np.where(
    df['Rider_Total_Rides'] > 0,
    df['Rider_Past_Cancels'] / df['Rider_Total_Rides'],
    0.0
)

# ---- 5) Driver history (only if a driver id column exists) ----
driver_id_col = None
for cand in ['Driver ID', 'Driver_ID', 'Driver Id', 'DriverID']:
    if cand in df.columns:
        driver_id_col = cand
        break

if driver_id_col is not None:
    df = df.sort_values([driver_id_col, 'Request_Datetime']).reset_index(drop=True)

    df['Driver_Total_Rides'] = df.groupby(driver_id_col).cumcount()

    df['Driver_Past_Cancels'] = (
        df.groupby(driver_id_col)['is_cancel']
          .shift(1)
          .fillna(0)
          .groupby(df[driver_id_col]).cumsum()
          .fillna(0)
    )

    df['Driver_Cancel_Rate'] = np.where(
        df['Driver_Total_Rides'] > 0,
        df['Driver_Past_Cancels'] / df['Driver_Total_Rides'],
        0.0
    )
else:
    # make empty (valid) columns so the feature list is stable
    df['Driver_Total_Rides'] = 0
    df['Driver_Cancel_Rate'] = 0.0

# ---- 6) Ensure cyclical encodings exist (from your earlier block) ----
if 'hour_sin' not in df.columns or 'hour_cos' not in df.columns:
    df['hour_sin'] = np.sin(2*np.pi*df['Hour']/24.0)
    df['hour_cos'] = np.cos(2*np.pi*df['Hour']/24.0)
if 'wday_sin' not in df.columns or 'wday_cos' not in df.columns:
    df['wday_sin'] = np.sin(2*np.pi*df['Weekday']/7.0)
    df['wday_cos'] = np.cos(2*np.pi*df['Weekday']/7.0)

# ---- 7) SAFE, pre-ride feature set (no post-ride leakage) ----
safe_num = [
    'Waiting Time',          # time before pickup — observable pre-ride
    'hour_sin','hour_cos',
    'wday_sin','wday_cos',
    'Rider_Total_Rides','Rider_Cancel_Rate',
    'Driver_Total_Rides','Driver_Cancel_Rate'
]

safe_cat = [
    'Vehicle Type',          # chosen/known at request
    'Pickup Location','Drop Location',
    'Payment Method'         # UPI/Cash/Wallet/Debit/Credit etc., chosen at request
]

# keep only columns that actually exist (avoids "not a column" errors)
safe_num = [c for c in safe_num if c in df.columns]
safe_cat = [c for c in safe_cat if c in df.columns]

X = df[safe_num + safe_cat]

# ---- 8) Pipeline ----
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler(with_mean=False))
        ]), safe_num),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), safe_cat)
    ]
)

clf = RandomForestClassifier(
    n_estimators=400,
    max_depth=None,
    n_jobs=-1,
    random_state=42,
    class_weight='balanced'
)

pipe = Pipeline([
    ('prep', preprocessor),
    ('rf', clf)
])

# ---- 9) Train / test split ----
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# ---- 10) Fit + evaluate ----
pipe.fit(X_train, y_train)
pred = pipe.predict(X_test)

print(confusion_matrix(y_test, pred))
print()
print(classification_report(y_test, pred, digits=3))

# ---- 11) Feature importances (readable names) ----
# Build output feature names matching the transformed matrix
num_names = safe_num
cat_names = []
if len(safe_cat):
    ohe = pipe.named_steps['prep'].named_transformers_['cat']
    ohe_feature_names = ohe.get_feature_names_out(safe_cat)
    cat_names = list(ohe_feature_names)

feature_names = num_names + cat_names
importances = pipe.named_steps['rf'].feature_importances_

fi = (pd.DataFrame({'feature': feature_names, 'importance': importances})
        .sort_values('importance', ascending=False)
        .head(20)
        .reset_index(drop=True))

print("\nTop feature importance (safe, pre-ride):")
display(fi)


[[22138     0]
 [   80  7298]]

              precision    recall  f1-score   support

           0      0.996     1.000     0.998     22138
           1      1.000     0.989     0.995      7378

    accuracy                          0.997     29516
   macro avg      0.998     0.995     0.996     29516
weighted avg      0.997     0.997     0.997     29516


Top feature importance (safe, pre-ride):


,feature,importance
0,Payment Method_nan,0.461706
1,Payment Method_UPI,0.153827
2,Waiting Time,0.139709
3,Payment Method_Cash,0.086062
4,Payment Method_Uber Wallet,0.041856
5,Payment Method_Credit Card,0.031996
6,Payment Method_Debit Card,0.024697
7,hour_cos,0.004775
8,hour_sin,0.004644
9,wday_sin,0.003376


In [28]:
df.groupby('Payment Method').is_cancel.mean()


Payment Method
Cash           0.0
Credit Card    0.0
Debit Card     0.0
UPI            0.0
Uber Wallet    0.0
Name: is_cancel, dtype: float64

In [29]:
df['Payment Method'].isna().mean(), df[df['Payment Method'].isna()]['is_cancel'].mean()


(np.float64(0.31988074264805527), np.float64(0.7814565327910523))

### Model 3 — Rider + Driver Historical Behavior (Leakage Identified)

This model introduced both rider and driver past-behavior features:

- `Rider_Total_Rides`
- `Rider_Cancel_Rate`
- `Driver_Total_Rides`
- `Driver_Cancel_Rate`

These were constructed using **only historical records** up to each request (no future information), which is correct and non-leaking.

#### What Went Wrong

During rider-sorted chronological modeling, we observed an unexpected performance jump:

- **Accuracy:** ~0.997
- **Precision/Recall:** ~0.99+ for both classes

This was far higher than prior models (~0.76–0.78 accuracy), which is a strong indicator of possible **data leakage**.

#### Root Cause

After investigation, we found that:

- **`Payment Method` had missing values**
- Missing values occurred **almost exclusively on canceled rides**
- This means the model could trivially learn:

> **If `Payment Method` is NaN → The ride was canceled.**

This is **not** a real behavioral signal — it's an **artifact of the dataset** and therefore constitutes **leakage**.

#### Verification

```python
df.groupby('Payment Method')['is_cancel'].mean()
df['Payment Method'].isna().mean(), df[df['Payment Method'].isna()]['is_cancel'].mean()


In [30]:
# =============================
# Model 3 (Leakage-Corrected)
# Rider + Driver History + Time Cycles + Vehicle + Location
# =============================

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

df = df.copy()

# Target already defined earlier, but ensure clean:
if 'Canceled' in df.columns:
    y = df['Canceled'].astype(int)
else:
    y = df['Booking Status'].astype(str).str.contains('cancel', case=False, na=False).astype(int)

df['is_cancel'] = y

# Ensure datetime exists:
assert 'Request_Datetime' in df.columns, "Run the datetime creation cell first."

# Sort for correct historical accumulation:
df = df.sort_values(['Customer ID', 'Request_Datetime']).reset_index(drop=True)

# ---- Rider History (no leakage) ----
df['Rider_Total_Rides'] = df.groupby('Customer ID').cumcount()

df['Rider_Past_Cancels'] = (
    df.groupby('Customer ID')['is_cancel']
      .shift(1)
      .fillna(0)
      .groupby(df['Customer ID']).cumsum()
)

df['Rider_Cancel_Rate'] = np.where(
    df['Rider_Total_Rides'] > 0,
    df['Rider_Past_Cancels'] / df['Rider_Total_Rides'],
    0.0
)

# ---- Driver History (if Driver ID exists) ----
driver_id_col = next((c for c in ['Driver ID','Driver_ID','Driver Id','DriverID'] if c in df.columns), None)

if driver_id_col:
    df = df.sort_values([driver_id_col, 'Request_Datetime']).reset_index(drop=True)

    df['Driver_Total_Rides'] = df.groupby(driver_id_col).cumcount()

    df['Driver_Past_Cancels'] = (
        df.groupby(driver_id_col)['is_cancel']
          .shift(1)
          .fillna(0)
          .groupby(df[driver_id_col]).cumsum()
    )

    df['Driver_Cancel_Rate'] = np.where(
        df['Driver_Total_Rides'] > 0,
        df['Driver_Past_Cancels'] / df['Driver_Total_Rides'],
        0.0
    )
else:
    df['Driver_Total_Rides'] = 0
    df['Driver_Cancel_Rate'] = 0.0

# ---- Ensure cyclical time encodings exist ----
if 'hour_sin' not in df.columns or 'hour_cos' not in df.columns:
    df['hour_sin'] = np.sin(2*np.pi*df['Hour']/24.0)
    df['hour_cos'] = np.cos(2*np.pi*df['Hour']/24.0)

if 'wday_sin' not in df.columns or 'wday_cos' not in df.columns:
    df['wday_sin'] = np.sin(2*np.pi*df['Weekday']/7.0)
    df['wday_cos'] = np.cos(2*np.pi*df['Weekday']/7.0)

# ===========================
# SAFE FEATURES (NO PAYMENT METHOD)
# ===========================

safe_num = [
    'Waiting Time',
    'hour_sin','hour_cos','wday_sin','wday_cos',
    'Rider_Total_Rides','Rider_Cancel_Rate',
    'Driver_Total_Rides','Driver_Cancel_Rate'
]

safe_cat = [
    'Vehicle Type',
    'Pickup Location','Drop Location'
    # Payment Method removed to eliminate leakage
]

safe_num = [c for c in safe_num if c in df.columns]
safe_cat = [c for c in safe_cat if c in df.columns]

X = df[safe_num + safe_cat]

# ---- Preprocessing + Random Forest ----
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler(with_mean=False))
        ]), safe_num),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), safe_cat)
    ]
)

clf = RandomForestClassifier(
    n_estimators=400,
    max_depth=None,
    n_jobs=-1,
    random_state=42,
    class_weight='balanced'
)

pipe = Pipeline([
    ('prep', preprocessor),
    ('rf', clf)
])

# ---- Train/Test ----
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

pipe.fit(X_train, y_train)
pred = pipe.predict(X_test)

print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred, digits=3))

# ---- Feature Importance ----
num_names = safe_num
ohe = pipe.named_steps['prep'].named_transformers_['cat']
cat_names = list(ohe.get_feature_names_out(safe_cat)) if len(safe_cat) else []

feature_names = num_names + cat_names
importances = pipe.named_steps['rf'].feature_importances_

fi = (pd.DataFrame({'feature': feature_names, 'importance': importances})
      .sort_values('importance', ascending=False)
      .head(20)
      .reset_index(drop=True))

print("\nTop feature importance (safe, pre-ride, leakage-corrected):")
display(fi)


[[21990   148]
 [ 6626   752]]
              precision    recall  f1-score   support

           0      0.768     0.993     0.867     22138
           1      0.836     0.102     0.182      7378

    accuracy                          0.770     29516
   macro avg      0.802     0.548     0.524     29516
weighted avg      0.785     0.770     0.695     29516


Top feature importance (safe, pre-ride, leakage-corrected):


,feature,importance
0,Waiting Time,0.221505
1,hour_cos,0.048377
2,hour_sin,0.048117
3,wday_sin,0.034456
4,wday_cos,0.023658
5,Vehicle Type_Auto,0.012792
6,Vehicle Type_Go Mini,0.011682
7,Vehicle Type_Bike,0.010472
8,Vehicle Type_Go Sedan,0.010132
9,Vehicle Type_Premier Sedan,0.009450


### Uber Cancellation Model — Rider + Driver History (No-Leakage)

### What Changed
- Added **history features** computed only from **past** records (no future info):
  - `Rider_Total_Rides`, `Rider_Cancel_Rate`
  - `Driver_Total_Rides`, `Driver_Cancel_Rate`
- Removed the leakage trigger we identified earlier (`Payment Method` missingness).
- Kept only **pre-ride** signals: `Waiting Time`, time cycles (`hour_sin`, `hour_cos`, `wday_sin`, `wday_cos`), `Vehicle Type`, and **non-leaking** location fields.

### Data Guards (Leakage Prevention)
- Sorted chronologically per **Customer** and per **Driver**.
- Used `.shift(1)` and cumulative sums for history rates so the **current outcome is never used**.
- Dropped/ignored any features only known **after** a ride completes (duration, distance, ratings, booking value, etc.).
- Removed `Payment Method` due to dataset artifact (NaN ≈ canceled).

### Performance (Hold-out)
- **Overall accuracy:** `0.770`
- **Per-class metrics**
  - **Class 0 (Completed)** — Precision `0.768`, Recall `0.993`, F1 `0.867`
  - **Class 1 (Canceled)** — Precision `0.836`, Recall `0.102`, F1 `0.182`
- **Macro-avg F1:** `~0.524`  
  (Realistic baseline with strict no-leakage constraints.)

### Top Features (Safe, Pre-Ride)
1. **Waiting Time**
2. **hour_cos**
3. **hour_sin**
4. **wday_sin**
5. **wday_cos**
6. **Vehicle Type** variants (Auto, Go Mini, Go Sedan, Bike, Premier, eBike, Uber XL)
7. Select **pickup/drop** locations (small but consistent signals)

### Interpretation
- **Time sensitivity** (hour/day cycles) and **expected wait** drive most of the signal.
- **History features** (rider/driver totals & cancel rates) help without leaking, but do **not** overwhelm the model.
- The model is excellent at confirming **completed** rides, and conservatively flags cancellations (low recall on Class 1 is expected without post-ride info or leaky proxies).

### Next Step — Cancellation Risk Scoring (Operational Use)

Now that we have a **validated, no-leakage model**, we can convert the model output 
(a probability of cancellation) into a **simple intervention scoring system**.

### Risk Score Scale
| Score Range | Interpretation | Recommended Action |
|------------|----------------|--------------------|
| **0.00 – 0.25** | Low Risk | No intervention needed. Display standard ride confirmation. |
| **0.25 – 0.60** | Medium Risk | Consider soft nudges: clarify ETA, show accurate wait time, highlight driver route. |
| **0.60 – 1.00** | High Risk | Trigger proactive stabilization actions: faster driver reassignment, incentive offers, or streamlined cancellation alternatives. |

### Why This Works
- The model captures **pre-ride decision pressure** (waiting time, time-of-day patterns).
- Risk scoring turns model output into **clear operational policy**.
- Interventions are **lightweight** and **do not require product redesign**.

### Example Implementation (Pseudo-Logic)



In [31]:
# ============================================
# Power BI export pack for Uber Cancellation model (no-leakage)
# Requires: df, X, y, pipe, X_train, X_test, y_train, y_test already defined
# ============================================
import os
import numpy as np
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

# 0) Output folder
EXPORT_DIR = "./exports_uber_cancel"
os.makedirs(EXPORT_DIR, exist_ok=True)

# 1) Get test-set predictions & probabilities
y_proba_test = pipe.predict_proba(X_test)[:, 1]
y_hat_test   = (y_proba_test >= 0.5).astype(int)
y_true_test  = y_test.copy()

# 2) Build a prediction table joined to original rows (only columns that exist)
id_cols = [c for c in [
    'Booking ID','Customer ID','Driver ID','Request_Datetime',
    'Pickup Location','Drop Location','Vehicle Type','Payment Method'
] if c in df.columns]

safe_context_cols = [c for c in [
    'Waiting Time','Hour','Weekday','hour_sin','hour_cos','wday_sin','wday_cos',
    'Rider_Total_Rides','Rider_Cancel_Rate','Driver_Total_Rides','Driver_Cancel_Rate'
] if c in df.columns]

pred_idx = X_test.index
pred_df = df.loc[pred_idx, id_cols + safe_context_cols].copy()
pred_df['y_true_cancel'] = y_true_test.values
pred_df['y_hat_cancel']  = y_hat_test
pred_df['p_cancel']      = y_proba_test

pred_path = os.path.join(EXPORT_DIR, "predictions_test.csv")
pred_df.to_csv(pred_path, index=False)

# 3) Feature importance table (readable names from pipeline)
num_names = []
cat_names = []
# Recover the feature lists used in the ColumnTransformer
prep = pipe.named_steps['prep']
if 'num' in prep.transformers_[0][0]:
    # Explicit num transformer named 'num'
    num_names = prep.transformers_[0][2]
else:
    # safer fallback (find by name)
    for name, trans, cols in prep.transformers_:
        if name == 'num':
            num_names = list(cols)

# Get OHE names for cat features
ohe = prep.named_transformers_['cat'] if 'cat' in prep.named_transformers_ else None
if ohe is not None:
    cat_names = list(ohe.get_feature_names_out(ohe.feature_names_in_))
feature_names = list(num_names) + list(cat_names)

rf = pipe.named_steps['rf']
importances = rf.feature_importances_
fi = (pd.DataFrame({'feature': feature_names, 'importance': importances})
        .sort_values('importance', ascending=False)
        .reset_index(drop=True))
fi_path = os.path.join(EXPORT_DIR, "feature_importance.csv")
fi.to_csv(fi_path, index=False)

# 4) Hourly / Weekday risk tables
risk_cols = []
if 'Hour' in pred_df.columns:
    risk_by_hour = (pred_df.groupby('Hour')
                    .agg(avg_p_cancel=('p_cancel','mean'),
                         n=('p_cancel','size'),
                         cancel_rate=('y_true_cancel','mean'))
                    .reset_index())
    risk_by_hour.to_csv(os.path.join(EXPORT_DIR, "risk_by_hour.csv"), index=False)
    risk_cols.append('Hour')

if 'Weekday' in pred_df.columns:
    risk_by_wday = (pred_df.groupby('Weekday')
                    .agg(avg_p_cancel=('p_cancel','mean'),
                         n=('p_cancel','size'),
                         cancel_rate=('y_true_cancel','mean'))
                    .reset_index())
    risk_by_wday.to_csv(os.path.join(EXPORT_DIR, "risk_by_weekday.csv"), index=False)
    risk_cols.append('Weekday')

if set(['Hour','Weekday']).issubset(pred_df.columns):
    risk_heat = (pred_df.pivot_table(index='Weekday', columns='Hour',
                                     values='p_cancel', aggfunc='mean')
                 .sort_index())
    risk_heat.to_csv(os.path.join(EXPORT_DIR, "risk_heatmap_weekday_by_hour.csv"))

# 5) Decile bands for simple risk segmentation (for KPI cards)
pred_df['risk_decile'] = pd.qcut(pred_df['p_cancel'], 10, labels=False, duplicates='drop')  # 0=lowest risk, 9=highest
deciles = (pred_df.groupby('risk_decile')
           .agg(avg_p_cancel=('p_cancel','mean'),
                cancel_rate=('y_true_cancel','mean'),
                n=('p_cancel','size'))
           .reset_index()
           .sort_values('risk_decile'))
deciles.to_csv(os.path.join(EXPORT_DIR, "risk_deciles.csv"), index=False)

# 6) Calibration curve bins (how well probs match reality)
cal = pred_df.copy()
cal['prob_bin'] = pd.cut(cal['p_cancel'], bins=np.linspace(0,1,11), include_lowest=True)
calib = (cal.groupby('prob_bin')
         .agg(avg_pred=('p_cancel','mean'),
              obs_rate=('y_true_cancel','mean'),
              n=('p_cancel','size'))
         .reset_index())
calib.to_csv(os.path.join(EXPORT_DIR, "calibration_bins.csv"), index=False)

# 7) Threshold sweep (choose operating point)
rows = []
for t in np.linspace(0,1,41):  # every 0.025
    y_hat_t = (y_proba_test >= t).astype(int)
    p, r, f1, _ = precision_recall_fscore_support(y_true_test, y_hat_t, average=None, labels=[0,1], zero_division=0)
    acc = accuracy_score(y_true_test, y_hat_t)
    rows.append({
        'threshold': t,
        'accuracy': acc,
        'prec_completed': p[0], 'rec_completed': r[0], 'f1_completed': f1[0],
        'prec_cancelled': p[1], 'rec_cancelled': r[1], 'f1_cancelled': f1[1]
    })
thresh_df = pd.DataFrame(rows)
thresh_df.to_csv(os.path.join(EXPORT_DIR, "threshold_metrics.csv"), index=False)

# 8) (Optional) Location aggregates if present (be careful about high cardinality)
for loc_col in ['Pickup Location','Drop Location']:
    if loc_col in pred_df.columns:
        tmp = (pred_df.groupby(loc_col)
               .agg(avg_p_cancel=('p_cancel','mean'),
                    cancel_rate=('y_true_cancel','mean'),
                    n=('p_cancel','size'))
               .reset_index()
               .sort_values('avg_p_cancel', ascending=False))
        tmp.to_csv(os.path.join(EXPORT_DIR, f"risk_by_{loc_col.replace(' ','_').lower()}.csv"), index=False)

print(f"Exports complete in: {EXPORT_DIR}")


Exports complete in: ./exports_uber_cancel


In [32]:
import os
os.getcwd()


'C:\\Users\\Matthias\\Downloads'

In [33]:
# Export full dataset with probability + risk band
df_export = df.copy()
df_export['prob_cancel'] = pipe.predict_proba(X)[:,1]

# Risk band thresholds
df_export['risk_band'] = pd.cut(
    df_export['prob_cancel'],
    bins=[0, 0.25, 0.60, 1.0],
    labels=['Low', 'Medium', 'High'],
    include_lowest=True
)

df_export.to_csv("uber_with_scores.csv", index=False)
print("Exported: uber_with_scores.csv")


Exported: uber_with_scores.csv
